### Todos:
### 1. preprocess the data to make it suitable for glove embeding:
### a. remove ' from words( don't -> dont)
### b. convert all letters to lowercase ( Hoping -> hoping)
### 2. find the frequency of each emoji to see if the dataset is balanced and balance the dataset

In [120]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import interesting_labels

In [2]:
import pandas

In [18]:
def get_data():
    header = ['status_id','tweet','label']
    data_set = pandas.read_csv('processed.txt',delimiter='\t',names = header)
    return data_set

def get_data_set():
    return global_data_set

def get_glove_embedding(glove): 
    return nn.Embedding.from_pretrained(glove.vectors)

def get_label_mapping():
    original_list = interesting_labels.wanted_list
    output_list = [i in range(len(original_list))]
    


In [122]:
len(interesting_labels.wanted_list)

82

In [4]:
global_data_set = get_data()

In [ ]:
glove = torchtext.vocab.GloVe(name="twitter.27B",dim=50)

In [19]:
glove_emb = get_glove_embedding(glove)

In [5]:
data_set = get_data_set()

In [6]:
len(data_set)

27857

In [7]:
type(data_set['tweet'][0])

str

In [8]:
data_set

,status_id,tweet,label
0,742407819496919041,Hoping I don't screw up this interview,1381
1,744101567981359105,I feel like a baby kangaroo stuck in it's moth...,1421
2,746426733834944512,girl ppl should be happy i even remembered her...,1381
3,743951261607690240,"Oh, the irony if Misha wins the Choice TV Scen...",1387
4,747298371967193088,"I miss you to , you so fake now",1397
5,743505228759187457,ever since I laid my eyes on Mikayla I knew sh...,1445
6,744146216263499776,i know,1420
7,744526330964762624,Bacolod please?!,1403
8,744327078019891201,ugh six months is a long time.,1447
9,747879438981947393,Things can change so quickly,1424


In [9]:
def split_tweet(tweet):
    tweet = tweet.replace(".", " . ") \
                 .replace(",", " , ") \
                 .replace(";", " ; ") \
                 .replace("?", " ? ")
    return tweet.split()

In [11]:
def get_tweet_words(glove_vector):
    train, valid, test = [],[],[]
    data_set = get_data_set()
    for i in range(len(data_set)):
        tweet = data_set['tweet'][i]
        if(type(tweet) != str):
            continue
        idxs = [glove_vector.stoi[w]        # lookup the index of word
                for w in split_tweet(tweet)
                if w in glove_vector.stoi] # keep words that has an embedding
        if not idxs: # ignore tweets without any word with an embedding
            continue
        idxs = torch.tensor(idxs) # convert list to pytorch tensor
        label = torch.tensor(data_set['label'][i]).long()
        if i % 5 < 3:
            train.append((idxs, label))
        elif i % 5 == 4:
            valid.append((idxs, label))
        else:
            test.append((idxs, label))
    return train, valid, test

In [12]:
train, valid, test = get_tweet_words(glove)

In [13]:
train[0]

(tensor([6456,   85,   53, 2706]), tensor(1381))

In [14]:
valid[0]

(tensor([ 292,   15,   16,    4,   15,   55, 1271,  110]), tensor(1397))

In [15]:
len(train)

16097

In [22]:
tweet_emb = glove_emb(train[0][0])
tweet_emb.shape

torch.Size([4, 50])

### Recurrent Neural Network Module

In [23]:
rnn_layer = nn.RNN(input_size=50,    # dimension of the input repr
                   hidden_size=50,   # dimension of the hidden units
                   batch_first=True) # input format is [batch_size, seq_len, repr_dim]

In [24]:
tweet_input = tweet_emb.unsqueeze(0) # add the batch_size dimension
h0 = torch.zeros(1, 1, 50)     # initial hidden layer
out, last_hidden = rnn_layer(tweet_input, h0)

In [25]:
print(out.shape)
print(last_hidden.shape)

torch.Size([1, 4, 50])
torch.Size([1, 1, 50])


In [26]:
out[:,-1,:]

tensor([[-0.0754, -0.0646,  0.4930, -0.0923, -0.5782, -0.0351,  0.4524,  0.6901,
         -0.3691, -0.3657, -0.6448, -0.1836, -0.1573,  0.0874, -0.2858,  0.1794,
          0.2335, -0.2746, -0.1823, -0.7109,  0.1043,  0.0144,  0.3161, -0.5042,
          0.4943, -0.3766,  0.2317,  0.1314, -0.5029,  0.4772,  0.1809,  0.5993,
         -0.5358, -0.3945, -0.4329,  0.3116, -0.0541,  0.1377, -0.2793,  0.4056,
         -0.1290,  0.6674,  0.2860, -0.3320,  0.6566, -0.0811, -0.6778,  0.7375,
         -0.6494, -0.4994]], grad_fn=<SliceBackward>)

### Building the model

In [113]:
class TweetRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TweetRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

model = TweetRNN(input_size=50, hidden_size=50, num_classes=len(interesting_labels.wanted_list))

In [28]:
data_set.nunique()

status_id    25977
tweet        25454
label           78
dtype: int64

In [46]:
train[0:16097][1]

(tensor([  264,    63,    11,   347, 46998,  2457,    35, 50306]),
 tensor(1421))

In [65]:
len(set([element[1].item() for element in train]))

78

In [67]:
import random

class TweetBatcher:
    def __init__(self, tweets, batch_size=32, drop_last=False):
        # store tweets by length
        self.tweets_by_length = {}
        for words, label in tweets:
            # compute the length of the tweet
            wlen = words.shape[0]
            # put the tweet in the correct key inside self.tweet_by_length
            if wlen not in self.tweets_by_length:
                self.tweets_by_length[wlen] = []
            self.tweets_by_length[wlen].append((words, label),)
         
        #  create a DataLoader for each set of tweets of the same length
        self.loaders = {wlen : torch.utils.data.DataLoader(
                                    tweets,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=drop_last) # omit last batch if smaller than batch_size
            for wlen, tweets in self.tweets_by_length.items()}
        
    def __iter__(self): # called by Python to create an iterator
        # make an iterator for every tweet length
        iters = [iter(loader) for loader in self.loaders.values()]
        while iters:
            # pick an iterator (a length)
            im = random.choice(iters)
            try:
                yield next(im)
            except StopIteration:
                # no more elements in the iterator, remove it
                iters.remove(im)

In [70]:
for i, (tweets, labels) in enumerate(TweetBatcher(train, drop_last=True)):
    print(tweets.shape, labels.shape)
print(i)

torch.Size([32, 1]) torch.Size([32])
torch.Size([32, 15]) torch.Size([32])
torch.Size([32, 12]) torch.Size([32])
torch.Size([32, 3]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 24]) torch.Size([32])
torch.Size([32, 14]) torch.Size([32])
torch.Size([32, 5]) torch.Size([32])
torch.Size([32, 26]) torch.Size([32])
torch.Size([32, 7]) torch.Size([32])
torch.Size([32, 25]) torch.Size([32])
torch.Size([32, 23]) torch.Size([32])
torch.Size([32, 23]) torch.Size([32])
torch.Size([32, 8]) torch.Size([32])
torch.Size([32, 15]) torch.Size([32])
torch.Size([32, 1]) torch.Size([32])
torch.Size([32, 16]) torch.Size([32])
torch.Size([32, 21]) torch.Size([32])
torch.Size([32, 21]) torch.Size([32])
torch.Size([32, 9]) torch.Size([32])
torch.Size([32, 23]) torch.Size([32])
torch.Size([32, 7]) torch.Size([32])
torch.Size([32, 9]) torch.Size([32])
torch.Size([32, 23]) torch.Size([32])
torch.Size([32, 21]) torch.Size([32])
torch.Size([32, 4]) torch.Size([32])
torch.Size([32, 6]) to

In [117]:
def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for tweets, labels in data_loader:
        output = model(tweets)
        pred = output.max(1, keepdim=True)[1]
        #print(pred)
        print(labels)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    print(correct)
    return float(correct) / float(total)

test_loader = TweetBatcher(test, batch_size=32, drop_last=False)
get_accuracy(model, test_loader)


tensor([1384, 1446, 1381, 1420, 1415, 1397, 1424, 1389])
tensor([1381, 1409, 1410, 1446, 1381, 1399, 1422, 1384, 1420, 1381, 1389, 1420,
        1620, 1424, 1392, 1381, 1399, 1397, 1381, 1381, 1384, 1381, 1400, 1384,
        1381, 1387, 1397, 1386, 1389, 1409, 1392, 1381])
tensor([1381, 1385, 1400, 1421, 1427, 1446, 1383, 1620, 1393, 1409, 1420, 1397,
        1400, 1392, 1395, 1406, 1381, 1381, 1423, 1381, 1447, 1383, 1399, 1413,
        1430, 1384, 1620, 1446, 1408, 1381, 1620, 1381])
tensor([1411, 1381, 1447,   78, 1381, 1620, 1381, 1428, 1423, 1446, 1383, 1381,
        1416, 1395, 1447, 1620, 1620, 1381, 1381, 1623, 1619, 1416, 1396, 1445,
        1379, 1398, 1413, 1384, 1389, 1380, 1389])
tensor([1445, 1381,   78, 1381, 1620, 1621, 1431, 1620, 1381, 1380, 1381, 1381,
        1424, 1381, 1389, 1380, 1384, 1400, 1382, 1388, 1447, 1392, 1430, 1383,
        1388, 1407, 1380, 1446, 1381, 1424, 1380, 1436])
tensor([1380, 1446, 1381, 1414, 1381, 1447, 1379,   78, 1428, 1420, 1381, 1412,
 

tensor([1397, 1389, 1447, 1424, 1424, 1388, 1381, 1447, 1617, 1433, 1620, 1424,
        1381, 1421, 1421, 1420, 1409, 1381, 1381, 1447, 1389, 1447, 1422, 1617,
        1394, 1446, 1445, 1424,   77, 1381, 1411, 1424])
tensor([1420, 1390, 1422, 1390, 1392, 1420, 1409, 1421, 1381, 1389, 1420, 1381,
        1381, 1447, 1424])
tensor([1403, 1392, 1397, 1382, 1386, 1381, 1424, 1389, 1406,   78, 1406, 1403,
        1403, 1392, 1389,   78, 1381, 1424, 1403, 1424, 1430, 1397, 1406, 1389,
        1394, 1381, 1392, 1406, 1420, 1423, 1447, 1394])
tensor([1430, 1445, 1431, 1447, 1392, 1392, 1396, 1620, 1383, 1381, 1392, 1399,
        1381, 1420, 1447, 1381,   77, 1431, 1618, 1381, 1395, 1386, 1389, 1418,
        1446, 1389, 1403, 1397, 1403, 1424, 1413,   77])
tensor([1389, 1413, 1424, 1428, 1399, 1394, 1380, 1384, 1424, 1398, 1430, 1420,
        1403, 1395, 1388, 1447, 1380, 1447, 1424, 1405, 1424, 1381, 1399, 1424,
        1397, 1393, 1445, 1424, 1420, 1403, 1620, 1401])
tensor([1421, 1407, 1381,

0.0

In [106]:
test_loader = TweetBatcher(test, batch_size=32, drop_last=True)

In [107]:
for tweets, labels in test_loader:
    print(tweets.shape)
    print(labels.shape)
    print(tweets[32])
    break

torch.Size([32, 10])
torch.Size([32])


IndexError: index 32 is out of bounds for dimension 0 with size 32

In [114]:
for i in range(100):
    test_loader = TweetBatcher(test, batch_size=32, drop_last=False)
    (get_accuracy(model, test_loader))


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
